In [6]:
from pandas import read_csv, Series, DataFrame, concat
import numpy as np
import pathlib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, accuracy_score, jaccard_score

In [7]:
#Number of recommendations set to 20
no_recom = 20
games_data = read_csv('processed_games_for_content-based.csv')
#users_data = read_csv('Content_Training_Testing/content_user_train.csv')
users_data = read_csv('Content_Training_Testing/content_user_train.csv')
#users_data = read_csv('steam_users_purchase_play.csv')
#Getting reviews information from the csv generated
games_review = read_csv('steam_games_review.csv', usecols=["name", "positive_review_percent"])
#Creating reverse map of indices and game names
games_indices = Series(games_data.index, index=games_data["name"]).drop_duplicates()
#Getting list of all unique games we have in games dataset
games_list = games_data['name'].unique()
#Creating new dataframe for recommendation
df_rec = list(map(str, range(1, no_recom+1)))
df_new = list(map(str, range(1, no_recom+1)))
#df_rec = ["user_id"] + df_rec
df_new = ["user_id"] + df_new

In [8]:
#Total unique games in training dataset
users_data.user_id.nunique()

11213

In [9]:
#Recommendation functions
#Function taking game titles (game name) and cosine similarity as input arguments and returns most similar games
def make_recommendation_by_sim(game_title, cosine_sim):
    if game_title not in games_list:
        return []
    #Index of the game that matches the name
    id = games_indices[game_title]
    if type(id) is Series:
        return []
    #Calculating pairwise similarity scores of all games with the game mentioned
    similarity_score = list(enumerate(cosine_sim[id]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    #Getting similarity scores of top 20 most similar games
    similarity_score = similarity_score[1:no_recom+1]
    #Getting game indices of the above 20 games with matching scores
    games_id =  [i[0] for i in similarity_score]
    #finally returning top most similar games
    return games_data['name'].iloc[games_id].tolist()


In [10]:
#Function taking user id , games user have and the games user already have as input 
#and making recommendations
def make_recommendation_for_user(user_id, game_list, games_of_user):
    if type(game_list) is not list or len(game_list) == 0:
        return DataFrame(data=[["NaN"] * no_recom], columns=df_rec)
    recom_reviews = games_review.loc[games_review['name'].isin(game_list)]
    recom_reviews = recom_reviews.loc[~recom_reviews['name'].isin(games_of_user)]
    recom_reviews = recom_reviews.sort_values(by='positive_review_percent', ascending=False)
    if len(recom_reviews) < no_recom:
        return DataFrame(data=[recom_reviews["name"].tolist() + ["NaN"] * (no_recom - len(recom_reviews.index))], columns=df_rec)
    else:
        return DataFrame(data=[recom_reviews["name"].tolist()[0:no_recom]], columns=df_rec)
    

In [11]:
#Function taking user id , games user have and the games user already have as input 
#and making recommendations for many users.
def make_recommendation_for_manyuser(user_id, game_list, games_of_user):
    if type(game_list) is not list or len(game_list) == 0:
        return DataFrame(data=[[user_id] + [""] * no_recom], columns=df_new)
    recom_reviews = games_review.loc[games_review['name'].isin(game_list)]
    recom_reviews = recom_reviews.loc[~recom_reviews['name'].isin(games_of_user)]
    recom_reviews = recom_reviews.sort_values(by='positive_review_percent', ascending=False)
    if len(recom_reviews) < no_recom:
        return DataFrame(data=[[user_id] + recom_reviews["name"].tolist() + [""] * (no_recom - len(recom_reviews.index))], columns=df_new)
    else:
        return DataFrame(data=[[user_id] + recom_reviews["name"].tolist()[0:no_recom]], columns=df_new)

In [12]:
#Function to generate recommendation for a user by a column feature
def make_recommendation_bycolumn(column_name, user_id):
    recom_by_userdata = DataFrame(columns=df_rec)
    #recom_by_userdata = list()
    games_data[column_name] = games_data[column_name].fillna('')
    #Calculating Cosine Similarity Matrix using the column feature
    count =  CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(games_data[column_name])
    similarity_matrix = cosine_similarity(count_matrix, count_matrix)
    #print(similarity_matrix)
    id = ''
    suggestion_list = list()
    games_user_has_list = list()
    for i, row in users_data.iterrows():
       if row["user_id"] == user_id:
         games_user_has_list.extend([row["game"]])
         suggestion_list.extend(make_recommendation_by_sim(row["game"], similarity_matrix))
         if len(suggestion_list) != 0:
            recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_user(user_id, suggestion_list, games_user_has_list)])
         if recom_by_userdata.shape[0] == 1:
          break
    print("Recommendation produced for user {} are:".format(user_id))
    #print(recom_by_userdata)
    #recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_user(user_id, suggestion_list, games_user_has_list)], ignore_index=True)
    recom_list = sum(recom_by_userdata.values.tolist(), [])
    if 'NaN' in recom_list:
      recom_list.remove('NaN')
    return recom_list

In [14]:
def generate_recommendation_output(column_name, location_file):
    recom_by_userdata = DataFrame(columns=df_new)

    # need to do some modification on data to make sure there is no NaN in column
    games_data[column_name] = games_data[column_name].fillna('')
    # Compute the Cosine Similarity matrix using the column
    count = CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(games_data[column_name])
    cosine_sim_matrix = cosine_similarity(count_matrix, count_matrix)

    id = ""
    listSuggestion = list()
    listGamesUserHas = list()

    # loop on all row and get recommendations for user
    for j, row in users_data.iterrows():
        if id != row["user_id"]:
            recom_by_userdata = concat([recom_by_userdata,
                                               make_recommendation_for_manyuser(id, listSuggestion, listGamesUserHas)],
                                              ignore_index=True)
            id = row["user_id"]
            listSuggestion = list()
            listGamesUserHas = list()
        listGamesUserHas.extend([row["game"]])
        listSuggestion.extend(make_recommendation_by_sim(row["game"], cosine_sim_matrix))

    # add the last element for the last user
    recom_by_userdata = concat([recom_by_userdata,
                                       make_recommendation_for_manyuser(id, listSuggestion, listGamesUserHas)],
                                      ignore_index=True)

    recom_by_userdata.to_csv(location_file, index=False)


In [15]:
recom_list = make_recommendation_bycolumn('popular_tags', 86540)
recom_list

Recommendation produced for user 86540 are:


['Amnesia: The Dark Descent',
 'F.E.A.R.',
 'Dead Space™ 2',
 'Alien: Isolation',
 'Layers of Fear',
 'Dead Space',
 'Max Payne',
 'Penumbra Overture',
 'Condemned: Criminal Origins',
 'resident evil 4 / biohazard 4',
 'Dungeon Nightmares II : The Memory',
 'Dark Sector',
 'The Evil Within',
 'Kholat',
 'Call of Cthulhu®: Dark Corners of the Earth',
 'Manhunt',
 'The Hat Man: Shadow Ward',
 'I Am Alive',
 'Amnesia: A Machine for Pigs',
 'Silent Hill Homecoming']

In [16]:
make_recommendation_bycolumn('genre_publisher_developer', 5250)

Recommendation produced for user 5250 are:


['Portal 2',
 'Left 4 Dead 2',
 'Half-Life 2',
 'Counter-Strike',
 'Counter-Strike: Source',
 'Half-Life',
 'Half-Life 2: Episode Two',
 'Half-Life: Opposing Force',
 'Left 4 Dead',
 'Half-Life 2: Episode One',
 'Half-Life: Blue Shift',
 'Day of Defeat: Source',
 'Half-Life 2: Deathmatch',
 'Day of Defeat',
 'Team Fortress Classic',
 'Deathmatch Classic',
 'Half-Life: Source',
 'Ricochet',
 'Half-Life Deathmatch: Source']

In [14]:
make_recommendation_bycolumn('genre_popular_tags_developer', 5250)

Recommendation produced for user 5250 are:


['Killing Floor',
 'Left 4 Dead 2',
 'Dead Space™ 2',
 'Team Fortress 2',
 'Synergy',
 'Warframe',
 'Arma 3',
 'Earth Defense Force: Insect Armageddon',
 "Tom Clancy's Rainbow Six® Vegas 2",
 'Resident Evil Revelations / Biohazard Revelations',
 'Halo: Spartan Strike',
 'Call of Duty®: Black Ops II',
 'Tom Clancy’s Splinter Cell Blacklist',
 'Alien Breed 2: Assault',
 'Primal Carnage',
 'ORION: Prelude',
 'Defiance',
 'Primal Carnage: Extinction',
 'God Mode',
 'Scourge: Outbreak']

In [17]:
#Function to generate recommendation for 20 users by a column feature
def make_recommendationforNusers(column_name):
    recom_by_userdata = DataFrame(columns=df_new)
    games_data[column_name] = games_data[column_name].fillna('')
    #Calculating Cosine Similarity Matrix using the column feature
    count =  CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(games_data[column_name])
    similarity_matrix = cosine_similarity(count_matrix, count_matrix)
    id = ''
    count = 0
    suggestion_list = list()
    games_user_has_list = list()
    for i, row in users_data.iterrows():
       if count == 20:
        break
       if row["user_id"] != id:   
         recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_manyuser(id, suggestion_list, games_user_has_list)])  
         id = row["user_id"]
         count += 1
         suggestion_list = list()
         games_user_has_list = list()
       games_user_has_list.extend([row["game"]])
       suggestion_list.extend(make_recommendation_by_sim(row["game"], similarity_matrix))
    recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_manyuser(id, suggestion_list, games_user_has_list)], ignore_index=True)
    return recom_by_userdata

In [18]:
df_genre = make_recommendationforNusers('genre')
df_genre

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,The Binding of Isaac: Rebirth,Psychonauts,Ori and the Blind Forest,Left 4 Dead 2,Dishonored,Counter-Strike: Source,BioShock Infinite,Vampire: The Masquerade - Bloodlines,Mafia II,...,Saints Row IV,Call of Duty® 4: Modern Warfare®,Mad Max,Call of Duty®: Modern Warfare® 2,Killing Floor 2,METAL GEAR SOLID V: THE PHANTOM PAIN,Far Cry 3,Wolfenstein: The New Order,Prey,Assassin’s Creed® IV Black Flag™
2,76767.0,Portal,Primordia,The Room,The Binding of Isaac: Rebirth,Left 4 Dead 2,Dishonored,FTL: Faster Than Light,Supreme Commander: Forged Alliance,Psychonauts,...,"Cook, Serve, Delicious!",Machinarium,Mini Metro,"Warhammer® 40,000: Dawn of War® - Game of the ...",Sid Meier's Civilization® V,Counter-Strike: Source,Creeper World 3: Arc Eternal,"Papers, Please",BioShock Infinite,Antichamber
3,86540.0,Keep Talking and Nobody Explodes,Freddi Fish 4: The Case of the Hogfish Rustler...,The Witcher® 3: Wild Hunt,Mount & Blade: Warband,Hotline Miami,Super Hexagon,The Binding of Isaac: Rebirth,Grim Legends 2: Song of the Dark Swan,Putt-Putt® Saves The Zoo,...,Castle Crashers®,Dust: An Elysian Tail,Stronghold Crusader HD,VVVVVV,Duck Game,Freddi Fish 2: The Case of the Haunted Schoolh...,Terraria,Psychonauts,Ori and the Blind Forest,Skullgirls
4,103360.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3
5,144736.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3
6,181212.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3
7,229911.0,The Binding of Isaac: Rebirth,Psychonauts,Left 4 Dead 2,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,Synergy,...,Echoes+,Warframe,Rexaura,Call of Duty® 4: Modern Warfare®,Half-Life 2: Update,Call of Duty®: Modern Warfare® 2,Killing Floor 2,Fortress Forever,Wolfenstein: The New Order,Teeworlds
8,298950.0,This War of Mine - War Child Charity DLC,fault - milestone two side:above,Keep Talking and Nobody Explodes,Castle Crashers - Pink Knight Pack,Primordia,The Room,Mount & Blade: Warband,Hylics,Choice of Robots,...,Hotline Miami,The Expendabros,BattleBlock Theater®,Don't Starve: Reign of Giants,Castle Crashers - Blacksmith Pack,The Witcher® 3: Wild Hunt,Super Hexagon,Putt-Putt® Saves The Zoo,Downwell,Crawl
9,299153.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3


In [19]:
df_tags = make_recommendationforNusers('popular_tags')
df_tags

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Battlefield: Bad Company 2 Vietnam,BattleBlock Theater®,Unreal Tournament: Game of the Year Edition,Killing Floor,Serious Sam Classic: The Second Encounter,Descent,Counter-Strike: Source,Quake III Arena,Half-Life 2: Episode Two,...,Half-Life: Opposing Force,BioShock™,System Shock 2,Shadow Warrior Classic (1997),Sanctum 2,Black Mesa,Serious Sam HD: The First Encounter,Team Fortress 2,Half-Life 2: Episode One,Descent 2
2,76767.0,The Witcher 3: Wild Hunt - Expansion Pass,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,The Witcher® 3: Wild Hunt,BattleBlock Theater®,Descent,Killing Floor,Half-Life 2,Serious Sam Classic: The Second Encounter,...,Quake III Arena,Half-Life 2: Episode Two,QUAKE,Shadow Warrior Classic (1997),System Shock 2,Half-Life: Opposing Force,Thomas Was Alone,Stronghold HD,Half-Life 2: Episode One,Team Fortress 2
3,86540.0,Battlefield: Bad Company 2 Vietnam,BattleBlock Theater®,Unreal Tournament: Game of the Year Edition,The Witcher® 3: Wild Hunt,planetarian ~the reverie of a little planet~,The Expendabros,Enigmatis 2: The Mists of Ravenwood,Botanicula,Serious Sam Classic: The Second Encounter,...,The Journey Down: Chapter Two,Descent,Killing Floor,Broforce,DARK SOULS™ II - Season Pass,S.T.A.L.K.E.R.: Call of Pripyat,Half-Life 2: Episode Two,Tomb Raider,Amnesia: The Dark Descent,QUAKE
4,103360.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Killing Floor,Serious Sam Classic: The Second Encounter,Descent,Half-Life 2,Quake III Arena,Half-Life 2: Episode Two,QUAKE,...,System Shock 2,Left 4 Dead,Alien Swarm,Shadow Warrior Classic (1997),Half-Life: Opposing Force,Half-Life 2: Episode One,Team Fortress 2,Serious Sam HD: The First Encounter,QUAKE II,Descent 2
5,144736.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Descent,Half-Life 2,Serious Sam Classic: The Second Encounter,Half-Life 2: Episode Two,QUAKE,Counter-Strike: Source,Quake III Arena,...,Shadow Warrior Classic (1997),System Shock 2,Half-Life 2: Episode One,Serious Sam HD: The First Encounter,QUAKE II,Descent 2,BioShock Infinite - Season Pass,Guns of Icarus Online,Arma 2: Operation Arrowhead,Tom Clancy's Ghost Recon®
6,181212.0,Battlefield: Bad Company 2 Vietnam,Portal,Unreal Tournament: Game of the Year Edition,Descent,Serious Sam Classic: The Second Encounter,Killing Floor,Half-Life 2: Episode Two,QUAKE,Quake III Arena,...,System Shock 2,Left 4 Dead,BioShock™,Shadow Warrior Classic (1997),Alien Swarm,Half-Life: Opposing Force,Black Mesa,Serious Sam HD: The First Encounter,Half-Life 2: Episode One,Team Fortress 2
7,229911.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Killing Floor,Descent,Serious Sam Classic: The Second Encounter,Half-Life 2: Episode Two,Quake III Arena,QUAKE,Counter-Strike: Source,...,Shadow Warrior Classic (1997),Alien Swarm,BioShock™,Left 4 Dead,Half-Life: Opposing Force,Serious Sam HD: The First Encounter,Black Mesa,Half-Life 2: Episode One,Descent 2,BioShock Infinite - Season Pass
8,298950.0,The Witcher 3: Wild Hunt - Expansion Pass,Tetrobot and Co.,Battlefield: Bad Company 2 Vietnam,The Witcher® 3: Wild Hunt,Hook,The Legend of Heroes: Trails in the Sky SC,The Binding of Isaac: Rebirth,Escape Goat,Primordia,...,BattleBlock Theater®,Don't Starve: Reign of Giants,planetarian ~the reverie of a little planet~,Unreal Tournament: Game of the Year Edition,Epic Battle Fantasy 4,Enigmatis 2: The Mists of Ravenwood,Outlast: Whistleblower DLC,Crypt of the NecroDancer,Hegemony Gold: Wars of Ancient Greece,Orcs Must Die!
9,299153.0,Battlefield: Bad Company 2 Vietnam,Portal,Unreal Tournament: Game of the Year Edition,Serious Sam Classic: The Second Encounter,Killing Floor,Quake III Arena,Counter-Strike: Source,Half-Life 2: Episode Two,Half-Life,...,Shadow Warrior Classic (1997),Half-Life: Opposing Force,BioShock™,Left 4 Dead,Ali

In [23]:
df_publisher = make_recommendationforNusers('publisher')
df_publisher

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Left 4 Dead 2,Counter-Strike: Source,Garry's Mod,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Team Fortress 2,Half-Life 2: Episode One,Half-Life: Blue Shift,...,Counter-Strike: Global Offensive,Half-Life 2: Deathmatch,Dota 2,Team Fortress Classic,Half-Life: Source,,,,,
2,76767.0,Keep Talking and Nobody Explodes,Portal,Left 4 Dead 2,Counter-Strike,Castle Crashers®,Half-Life 2,Euro Truck Simulator 2,Garry's Mod,Quake III Arena,...,Tomb Raider,Half-Life 2: Episode Two,QUAKE,Left 4 Dead,The Jackbox Party Pack 2,XCOM: Enemy Unknown,Half-Life: Opposing Force,Race The Sun,Mount Your Friends,Team Fortress 2
3,86540.0,Keep Talking and Nobody Explodes,The Expendabros,Sam & Max 105: Reality 2.0,Mount & Blade: Warband,Hotline Miami,Sam & Max 102: Situation: Comedy,The Wolf Among Us,Broforce,Don't Starve,...,Serious Sam Classic: The Second Encounter,Downwell,Garry's Mod,Half-Life 2: Episode Two,Counter-Strike: Source,Sid Meier's Civilization V: Brave New World,Sam & Max 203: Night of the Raving Dead,BioShock Infinite,SOMA,Hatoful Boyfriend
4,103360.0,Portal 2,Left 4 Dead 2,Half-Life 2,Counter-Strike: Source,Garry's Mod,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Team Fortress 2,...,Half-Life: Blue Shift,Day of Defeat: Source,Counter-Strike: Global Offensive,Half-Life 2: Deathmatch,Dota 2,Half-Life: Source,,,,
5,144736.0,Portal 2,Half-Life 2,Left 4 Dead 2,Counter-Strike: Source,Garry's Mod,Half-Life 2: Episode Two,Half-Life: Opposing Force,Left 4 Dead,Team Fortress 2,...,Half-Life: Blue Shift,Day of Defeat: Source,Counter-Strike: Global Offensive,Half-Life 2: Deathmatch,Dota 2,Team Fortress Classic,Half-Life: Source,,,
6,181212.0,Portal 2,Left 4 Dead 2,Counter-Strike,Counter-Strike: Source,Garry's Mod,Half-Life 2: Episode Two,Half-Life,Half-Life: Opposing Force,Left 4 Dead,...,Half-Life 2: Episode One,Half-Life: Blue Shift,Day of Defeat: Source,Counter-Strike: Global Offensive,Half-Life 2: Deathmatch,Dota 2,Half-Life: Source,,,
7,229911.0,Portal 2,Left 4 Dead 2,Counter-Strike: Source,Garry's Mod,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Half-Life 2: Episode One,Half-Life: Blue Shift,...,Counter-Strike: Global Offensive,Half-Life 2: Deathmatch,Dota 2,Half-Life: Source,,,,,,
8,298950.0,Ori and the Blind Forest (Original Soundtrack),The Witcher 3: Wild Hunt - Expansion Pass,Keep Talking and Nobody Explodes,One Finger Death Punch,Sam & Max 105: Reality 2.0,Mount & Blade: Warband,Hotline Miami,The Expendabros,Outlast: Whistleblower DLC,...,South Park™: The Stick of Truth™,Don't Starve: Reign of Giants,Sam & Max 102: Situation: Comedy,Broforce,Stronghold Crusader HD,Ori and the Blind Forest,Don't Starve: Shipwrecked,Dust: An Elysian Tail,Orcs Must Die!,Crypt of the NecroDancer
9,299153.0,Portal 2,Left 4 Dead 2,Counter-Strike: Source,Garry's Mod,Half-Life,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Team Fortress 2,...,Half-Life: Blue Shift,Day of Defeat: Source,Counter-Strike: Global Offensive,Half-Life 2: Deathmatch,Dota 2,Half-Life: Source,,,,


In [24]:
df_gamedetail = make_recommendationforNusers('game_details')
df_gamedetail

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Battlefield: Bad Company 2 Vietnam,Gunpoint,Escape Goat,Outlast,Crypt of the NecroDancer,Amnesia: The Dark Descent,"Warhammer® 40,000: Dawn of War® - Dark Crusade",Creeper World 3: Arc Eternal,The Talos Principle,...,LISA,Half-Life 2: Episode Two,POSTAL 2,Race The Sun,Left 4 Dead,"Warhammer® 40,000: Dawn of War® - Soulstorm",Half-Life: Opposing Force,Heretic: Shadow of the Serpent Riders,Black Mesa,Shadow Warrior
2,76767.0,Battlefield: Bad Company 2 Vietnam,Outlast: Whistleblower DLC,Gunpoint,Serious Sam Classic: The Second Encounter,Counter-Strike,Outlast,Broforce,Hexcells,Crypt of the NecroDancer,...,Freedom Planet,Bastion,Machinarium,Bejeweled® 3,The Talos Principle,POSTAL 2,Creeper World 3: Arc Eternal,"Warhammer® 40,000: Dawn of War® - Dark Crusade",Quake III Arena,Axiom Verge
3,86540.0,Battlefield: Bad Company 2 Vietnam,Super Hexagon,The Witcher® 3: Wild Hunt,Primordia,Gunpoint,Blackwell Deception,Escape Goat,Indie Game: The Movie,Crypt of the NecroDancer,...,To the Moon,Botanicula,Outlast,Dust: An Elysian Tail,Broforce,Cave Story+,Super Amazing Wagon Adventure,Quake III Arena,Saints Row: The Third,Free to Play
4,103360.0,Battlefield: Bad Company 2 Vietnam,Quake III Arena,"Warhammer® 40,000: Dawn of War® - Dark Crusade","Warhammer® 40,000: Dawn of War® - Soulstorm",Half-Life: Opposing Force,Heretic: Shadow of the Serpent Riders,STAR WARS™ Republic Commando™,Return to Castle Wolfenstein,Swords and Soldiers HD,...,Moonbase Alpha,Red Faction,Sword of the Stars: Complete Collection,DOOM 3,Half-Life 2: Deathmatch,Spectromancer,Call of Duty®: Black Ops,Squad,Mitos.is: The Game,The Mean Greens - Plastic Warfare
5,144736.0,Battlefield: Bad Company 2 Vietnam,Quake III Arena,"Warhammer® 40,000: Dawn of War® - Dark Crusade","Warhammer® 40,000: Dawn of War® - Soulstorm",Half-Life: Opposing Force,Heretic: Shadow of the Serpent Riders,STAR WARS™ Republic Commando™,Return to Castle Wolfenstein,Swords and Soldiers HD,...,Moonbase Alpha,Red Faction,Sword of the Stars: Complete Collection,DOOM 3,Half-Life 2: Deathmatch,Spectromancer,Call of Duty®: Black Ops,Squad,Mitos.is: The Game,Team Fortress Classic
6,181212.0,Battlefield: Bad Company 2 Vietnam,Outlast,Counter-Strike,LISA,Half-Life,Half-Life: Opposing Force,Shadow Warrior,STAR WARS™ Republic Commando™,Deus Ex: Human Revolution - Director's Cut,...,Swords and Soldiers HD,The Banner Saga,Moonbase Alpha,Call of Duty®: Modern Warfare® 2,Sword of the Stars: Complete Collection,Half-Life 2: Deathmatch,Spectromancer,Darkest Dungeon®,Call of Duty®: Black Ops,Squad
7,229911.0,Battlefield: Bad Company 2 Vietnam,Portal 2,Escape Goat,Left 4 Dead 2,Killing Floor,Outlast,Amnesia: The Dark Descent,"Warhammer® 40,000: Dawn of War® - Dark Crusade",Quake III Arena,...,Half-Life 2: Episode Two,LISA,"Warhammer® 40,000: Dawn of War® - Soulstorm",Left 4 Dead,Half-Life: Opposing Force,STAR WARS™ Republic Commando™,Black Mesa,Estranged: Act I,Heretic: Shadow of the Serpent Riders,Shadow Warrior
8,298950.0,Battlefield: Bad Company 2 Vietnam,One Finger Death Punch,Escape Goat,TIS-100,Blackwell Deception,The Room,Primordia,South Park™: The Stick of Truth™,Mount & Blade: Warband,...,BattleBlock Theater®,The Expendabros,The Legend of Heroes: Trails in the Sky SC,Unreal Tournament: Game of the Year Edition,Freddi Fish 4: The Case of the Hogfish Rustler...,Bullet Heaven 2,The Binding of Isaac: Rebirth,The Witcher® 3: Wild Hunt,Super Hexagon,Botanicula
9,299153.0,Battlefield: Bad Company 2 Vietnam,Outlast,LISA,Half-Life,Half-Life: Opposing Force,Shadow Warrior,STAR WARS™ Republic Commando™,Deus Ex: Human Revolution - Director's Cut,Apotheon,...,The Banner Saga,Moonbase Alpha,Call of Duty®: Modern Warfare® 2,Sword of the Stars: Complete Collection,Half-Life 2: Deathmatch,Spectromancer,Darkest Dungeon®,Call of Duty®: Black Ops,Squad,Warhammer: End Times - Vermintide


In [20]:
df_gen_pub_dev = make_recommendationforNusers('genre_publisher_developer')
df_gen_pub_dev

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Left 4 Dead 2,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Team Fortress 2,Half-Life 2: Episode One,Half-Life: Blue Shift,Day of Defeat: Source,...,Half-Life 2: Deathmatch,Dota 2,Team Fortress Classic,A Walk in the Dark,Half-Life: Source,Half-Life Deathmatch: Source,,,,
2,76767.0,Portal,The Journey Down: Chapter Two,Counter-Strike,Half-Life 2,Euro Truck Simulator 2,Left 4 Dead 2,Quake III Arena,Tomb Raider,Half-Life 2: Episode Two,...,XCOM: Enemy Unknown,Mount Your Friends,Half-Life: Opposing Force,Left 4 Dead,Half-Life 2: Episode One,Euro Truck Simulator 2 - Ice Cold Paint Jobs Pack,Team Fortress 2,Mafia II,LIMBO,Alien: Isolation
3,86540.0,Keep Talking and Nobody Explodes,The Room,Sam & Max 105: Reality 2.0,Hotline Miami,Sam & Max 102: Situation: Comedy,The Wolf Among Us,Mount & Blade: Warband,Broforce,Castle Crashers®,...,Downwell,GemCraft - Chasing Shadows,Serious Sam Classic: The Second Encounter,The Talos Principle,SOMA,BioShock Infinite,Sid Meier's Civilization® V,Half-Life 2: Episode Two,Sid Meier's Civilization V: Brave New World,Counter-Strike: Source
4,103360.0,Portal 2,Left 4 Dead 2,Half-Life 2,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,...,Day of Defeat: Source,Half-Life 2: Deathmatch,Half-Life: Source,Half-Life Deathmatch: Source,,,,,,
5,144736.0,Portal 2,Left 4 Dead 2,Half-Life 2,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,...,Day of Defeat: Source,Half-Life 2: Deathmatch,Team Fortress Classic,Half-Life: Source,Half-Life Deathmatch: Source,,,,,
6,181212.0,Portal 2,Left 4 Dead 2,Counter-Strike,Counter-Strike: Source,Half-Life,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,...,Half-Life: Blue Shift,Day of Defeat: Source,Half-Life 2: Deathmatch,Half-Life: Source,Half-Life Deathmatch: Source,,,,,
7,229911.0,Portal 2,Left 4 Dead 2,Counter-Strike: Source,Half-Life 2: Episode Two,Alien Swarm,Left 4 Dead,Half-Life: Opposing Force,Half-Life 2: Episode One,Echoes+,...,Super Crate Box,Vanguard Princess Director's Cut,Half-Life: Blue Shift,Day of Defeat: Source,Time Clickers,Half-Life 2: Deathmatch,Counter-Strike: Global Offensive,Dota 2,Brawlhalla,Codename CURE
8,298950.0,Ori and the Blind Forest (Original Soundtrack),Shelter 2 Soundtrack,Europa Universalis III: Divine Wind,One Finger Death Punch,Battlefield: Bad Company 2 Vietnam,Keep Talking and Nobody Explodes,Luxor Evolved,Sam & Max 105: Reality 2.0,The Binding of Isaac: Rebirth,...,Sam & Max 102: Situation: Comedy,Freedom Planet - Official Soundtrack,Hotline Miami,Don't Starve: Reign of Giants,Unreal Tournament: Game of the Year Edition,Outlast: Whistleblower DLC,TIS-100,The Expendabros,Mount & Blade: Warband,Euro Truck Simulator 2
9,299153.0,Portal 2,Left 4 Dead 2,Counter-Strike: Source,Half-Life,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,...,Day of Defeat: Source,Half-Life 2: Deathmatch,Half-Life: Source,Ricochet,Half-Life Deathmatch: Source,,,,,


In [21]:
df_tags_dev_gen = make_recommendationforNusers('genre_popular_tags_developer')
df_tags_dev_gen

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Battlefield: Bad Company 2 Vietnam,BattleBlock Theater®,Unreal Tournament: Game of the Year Edition,Serious Sam Classic: The Second Encounter,Killing Floor,Left 4 Dead 2,Counter-Strike: Source,Half-Life 2: Episode Two,QUAKE,...,BioShock™,Half-Life: Opposing Force,F.E.A.R.,Dead Space™ 2,Mass Effect 2,Black Mesa,Vampire: The Masquerade - Bloodlines,Sanctum 2,Team Fortress 2,Half-Life 2: Episode One
2,76767.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,The Expendabros,The Witcher® 3: Wild Hunt,BattleBlock Theater®,Mount & Blade: Warband,Serious Sam Classic: The Second Encounter,Left 4 Dead 2,Dishonored,...,Killing Floor,Quake III Arena,QUAKE,Half-Life 2: Episode Two,Dead Space™ 2,Papo & Yo,Guacamelee! Super Turbo Championship Edition,F.E.A.R.,Mass Effect 2,Half-Life: Opposing Force
3,86540.0,Battlefield: Bad Company 2 Vietnam,The Room,Enigmatis 2: The Mists of Ravenwood,planetarian ~the reverie of a little planet~,Grim Legends 2: Song of the Dark Swan,BattleBlock Theater®,Mount & Blade: Warband,The Journey Down: Chapter Two,Terraria,...,Serious Sam Classic: The Second Encounter,Duck Game,Broforce,Botanicula,Castle Crashers®,QUAKE,SOMA,Quake III Arena,S.T.A.L.K.E.R.: Call of Pripyat,Half-Life 2: Episode Two
4,103360.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Half-Life 2,Counter-Strike: Source,Quake III Arena,Half-Life 2: Episode Two,QUAKE,Left 4 Dead,Half-Life: Opposing Force,...,Alien Swarm,F.E.A.R.,Half-Life 2: Episode One,Team Fortress 2,QUAKE II,Call of Duty: World at War,Descent 2,Call of Duty®,Tom Clancy's Splinter Cell Chaos Theory®,Guns of Icarus Online
5,144736.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Half-Life 2,Quake III Arena,Half-Life 2: Episode Two,QUAKE,Counter-Strike: Source,Mass Effect 2,F.E.A.R.,...,Half-Life 2: Episode One,QUAKE II,Tom Clancy's Splinter Cell Chaos Theory®,Descent 2,Call of Duty®,Call of Duty: World at War,Delta Force: Black Hawk Down,Arma 2: Operation Arrowhead,Call of Duty® 2,STAR WARS™ Jedi Knight II - Jedi Outcast™
6,181212.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Counter-Strike,Half-Life 2: Episode Two,QUAKE,Quake III Arena,Half-Life,BioShock™,Half-Life: Opposing Force,...,Left 4 Dead,Alien Swarm,Mass Effect 2,Black Mesa,Half-Life 2: Episode One,Team Fortress 2,QUAKE II,Descent 2,Tom Clancy's Splinter Cell Chaos Theory®,Call of Duty: World at War
7,229911.0,Battlefield: Bad Company 2 Vietnam,Portal 2,Unreal Tournament: Game of the Year Edition,Counter-Strike: Source,Half-Life 2: Episode Two,QUAKE,Quake III Arena,Half-Life: Opposing Force,BioShock™,...,Left 4 Dead,F.E.A.R.,Alien Swarm,Half-Life 2: Episode One,Black Mesa,Vampire: The Masquerade - Bloodlines,Call of Duty®,QUAKE II,BioShock Infinite - Season Pass,Tom Clancy's Splinter Cell Chaos Theory®
8,298950.0,The Witcher 3: Wild Hunt - Expansion Pass,Battlefield: Bad Company 2 Vietnam,The Legend of Heroes: Trails in the Sky SC,planetarian ~the reverie of a little planet~,Eternal Senia,Tex Murphy: Under a Killing Moon,Outlast: Whistleblower DLC,Unreal Tournament: Game of the Year Edition,The Binding of Isaac: Rebirth,...,Primordia,Epic Battle Fantasy 4,Enigmatis 2: The Mists of Ravenwood,Don't Starve: Reign of Giants,Grim Legends 2: Song of the Dark Swan,BattleBlock Theater®,The Witcher® 3: Wild Hunt,Luxor Evolved,Escape Goat,Killing Floor
9,299153.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Counter-Strike: Source,Half-Life,Half-Life 2: Episode Two,QUAKE,Quake III Arena,Left 4 Dead,BioShock™,...,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,Team Fortress 2,Black Mesa,Tom Clancy's Splinter Cell Chaos Theory®,QUAKE II,Call of Duty®,Descent 2,Call of Duty: World at War


In [25]:
df_gen_tags_details = make_recommendationforNusers('genre_popular_tags_game_details')
df_gen_tags_details

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Battlefield: Bad Company 2 Vietnam,BattleBlock Theater®,Gunpoint,Unreal Tournament: Game of the Year Edition,Left 4 Dead 2,Crypt of the NecroDancer,Psychonauts,Killing Floor,BioShock Infinite,...,Half-Life 2: Episode Two,Tomb Raider,Amnesia: The Dark Descent,Total Annihilation,Quake III Arena,STAR WARS™ Jedi Knight - Jedi Academy™,Left 4 Dead,Outcast 1.1,Half-Life: Opposing Force,Serious Sam HD: The Second Encounter
2,76767.0,Battlefield: Bad Company 2 Vietnam,BattleBlock Theater®,Unreal Tournament: Game of the Year Edition,The Expendabros,The Witcher® 3: Wild Hunt,Counter-Strike,Dishonored,Broforce,Left 4 Dead 2,...,Mark of the Ninja,Castle Crashers®,Serious Sam Classic: The Second Encounter,Killing Floor,QUAKE,Total Annihilation,Quake III Arena,Serious Sam HD: The Second Encounter,Orcs Must Die! 2,Left 4 Dead
3,86540.0,Battlefield: Bad Company 2 Vietnam,South Park™: The Stick of Truth™,planetarian ~the reverie of a little planet~,Grim Legends 2: Song of the Dark Swan,Unreal Tournament: Game of the Year Edition,Tex Murphy: Under a Killing Moon,BattleBlock Theater®,Gunpoint,The Journey Down: Chapter Two,...,Psychonauts,Botanicula,Crypt of the NecroDancer,Crawl,Terraria,Castle Crashers®,Killing Floor,Duck Game,Tomb Raider,SOMA
4,103360.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Total Annihilation,QUAKE,Quake III Arena,STAR WARS™ Jedi Knight - Jedi Academy™,Half-Life: Opposing Force,Tom Clancy's Ghost Recon® Desert Siege™,STAR WARS™ Republic Commando™,...,Call of Duty®,Descent 2,Call of Duty: World at War,Return to Castle Wolfenstein,Delta Force: Black Hawk Down,STAR WARS™ Jedi Knight II - Jedi Outcast™,Call of Duty® 4: Modern Warfare®,Call of Duty® 2,BioShock® 2,Half-Life: Blue Shift
5,144736.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Total Annihilation,Quake III Arena,QUAKE,Half-Life: Opposing Force,STAR WARS™ Jedi Knight - Jedi Academy™,STAR WARS™ Republic Commando™,Tom Clancy's Ghost Recon® Desert Siege™,...,Descent 2,Call of Duty: World at War,QUAKE II,Delta Force: Black Hawk Down,Return to Castle Wolfenstein,Call of Duty® 4: Modern Warfare®,STAR WARS™ Jedi Knight II - Jedi Outcast™,Call of Duty® 2,BioShock® 2,Call of Duty®: Modern Warfare® 2
6,181212.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Counter-Strike,BioShock Infinite,Tomb Raider,Quake III Arena,Half-Life,Half-Life 2: Episode Two,QUAKE,...,STAR WARS™ Republic Commando™,Black Mesa,Half-Life 2: Episode One,Descent 2,Hacknet,QUAKE II,Call of Duty®,Call of Duty: World at War,Deus Ex: Human Revolution - Director's Cut,Delta Force: Black Hawk Down
7,229911.0,Battlefield: Bad Company 2 Vietnam,Portal 2,Gunpoint,Unreal Tournament: Game of the Year Edition,Psychonauts,Left 4 Dead 2,Killing Floor,BioShock Infinite,QUAKE,...,Tomb Raider,Amnesia: The Dark Descent,Quake III Arena,Total Annihilation,Garry's Mod,Outcast 1.1,STAR WARS™ Jedi Knight - Jedi Academy™,Left 4 Dead,Half-Life: Opposing Force,Half-Life 2: Episode One
8,298950.0,The Witcher 3: Wild Hunt - Expansion Pass,Battlefield: Bad Company 2 Vietnam,Enigmatis 2: The Mists of Ravenwood,The Legend of Heroes: Trails in the Sky SC,The Witcher® 3: Wild Hunt,Unreal Tournament: Game of the Year Edition,South Park™: The Stick of Truth™,Don't Starve: Reign of Giants,Primordia,...,The Binding of Isaac: Rebirth,BattleBlock Theater®,Outlast: Whistleblower DLC,Luxor Evolved,Hook,planetarian ~the reverie of a little planet~,Epic Battle Fantasy 4,Tex Murphy: Under a Killing Moon,Castle Crashers - Blacksmith Pack,Escape Goat
9,299153.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,BioShock Infinite,Quake III Arena,QUAKE,Half-Life,Half-Life 2: Episode Two,Tomb Raider,Outcast 1.1,...,Tom Clancy's Ghost Recon® Desert Siege™,Half-Life 2: Episode One,Black Mesa,Descent 2,Call of Duty®,Call of Duty: Wor

Generation of test files that contains recommendations produced for User-Id with genre, popular tags, developer, publisher, game-details and combination of these attributes as input.

In [22]:

generate_recommendation_output('genre', pathlib.Path('Test_Files/genre.csv'))

In [18]:
generate_recommendation_output('popular_tags', pathlib.Path('Test_Files/tags.csv'))

In [19]:
generate_recommendation_output('publisher', pathlib.Path('Test_Files/publisher.csv'))

In [20]:
generate_recommendation_output('game_details', pathlib.Path('Test_Files/game_details.csv'))

In [21]:
generate_recommendation_output('genre_publisher_developer', pathlib.Path('Test_Files/genre_pub_dev.csv'))

In [22]:
generate_recommendation_output('genre_popular_tags_developer', pathlib.Path('Test_Files/genre_tags_dev.csv'))

In [23]:
generate_recommendation_output('genre_popular_tags_game_details', pathlib.Path('Test_Files/genre_tags_gamedetail.csv'))